In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.firefox.options import Options
import pandas as pd

In [2]:
browser = Chrome() #open chrome in selenium
browser.get('https://meduza.io/specials/native') # open page with a lot of urls to ads

In [3]:
list_ = []
links = browser.find_elements_by_xpath('//*[@href]') # find all urls on page
for ii in links:
    list_.append(ii.get_attribute('href')) #append all hrefs to a list

In [4]:
meduza_ads = []
for item in list_:
    if item[:23] == 'https://meduza.io/cards' or item[:25] == 'https://meduza.io/feature': # get rid of non-ad urls
        meduza_ads.append(item)
meduza_ads_ = meduza_ads[1:-5] # get rid of not important urls like: how meduza's using coockie, etc

In [5]:
len(meduza_ads_) # check is len of meduza_ads long enough

386

In [6]:
p = [] #create list
df = pd.DataFrame() #create df
for item in meduza_ads_:
    r = requests.get(str(item)) #open url 
    soup = BeautifulSoup(r.text, 'html5lib') # get html as text
    doc = {} #create dictionary
    if item[:23] == 'https://meduza.io/cards': # get only values that firsst values equals to https://meduza.io/cards
        doc['text'] = soup.find('div', {'class' : 'CardMaterial-body'}).text #get text from card url
        text = doc['text']
    elif not soup.find('div', {'class' : 'Lead'}):
        continue
    else:
        doc['text'] = soup.find('div', {'class' : 'Lead'}).text #get text from feature url
        doc['txt'] = soup.find('div', {'class' : 'Body'}).text #get text from feature url
        text = ''.join((doc['text'], doc['txt'])) # join 2 string to 1
    p.append(text) #every iterate we append all texts to list
    df = pd.DataFrame.from_records(({"value": i} for i in p)) #convert list to dataframe

In [7]:
df.to_csv('parsed_data.csv', index=False) #save df to csv

In [8]:
a = pd.read_csv('parsed_data.csv') # check csv